##### Uruchamianie wszystkiego co działa na Macu:
YOLO, LVIS, OWL-ViT, lekki DETR (jeżeli chcesz).
Zapis w jednolitym formacie CSV.


### 0. Importy, ścieżki i konfiguracja

In [84]:
from pathlib import Path
from typing import List
import pandas as pd

"""
Przygotowanie środowiska dla benchmarku detektorów uruchamianych na Macu:
- YOLO (model własny),
- LVIS (gotowy model),
- OWL-ViT (open-vocabulary),
- opcjonalnie DETR.

Założenia:
- 08_eval_setup.ipynb utworzył plik outputs/detector_eval/eval_file_list.csv
  z kolumną 'file_name'.
- Obrazy testowe fizycznie znajdują się w katalogu data/test_scenes
  względem PROJECT_ROOT.
"""

PROJECT_ROOT = Path.cwd().resolve()
DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = DATA_DIR / "test_scenes"

EVAL_OUTPUT_DIR = PROJECT_ROOT / "outputs" / "detector_eval"
EVAL_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

TEST_LIST_CSV = EVAL_OUTPUT_DIR / "eval_file_list.csv"
df_files = pd.read_csv(TEST_LIST_CSV)
file_list: List[str] = df_files["file_name"].astype(str).tolist()

### 1. Wspólna funkcja pomocnicza: format wyników

In [85]:
"""
Definiuje funkcję pomocniczą tworzącą pusty DataFrame w jednolitym formacie.

Kolumny:
- file_name, model, prompt, class_raw, class_norm, score,
- x_min, y_min, x_max, y_max.
"""

DETECTIONS_COLUMNS = [
    "file_name",
    "model",
    "prompt",
    "class_raw",
    "class_norm",
    "score",
    "x_min",
    "y_min",
    "x_max",
    "y_max",
]


def empty_detections_df() -> pd.DataFrame:
    """Zwraca pusty DataFrame z kolumnami DETECTIONS_COLUMNS."""
    return pd.DataFrame(columns=DETECTIONS_COLUMNS)

### 2. Funkcja run_yolo (szkielet)

In [86]:
from ultralytics import YOLO

"""
YOLO – model trenowany na Syrenę i Malucha.
"""

BEST_RUN = "syrena_maluch_fast2"  # <- tu wstaw wybrany run
YOLO_MODEL_PATH = PROJECT_ROOT / "outputs" / "yolo_cars" / BEST_RUN / "weights" / "best.pt"

# indeksy klas w Twoim modelu YOLO
YOLO_IDX_TO_CLASS_NORM = {
    0: "maluch",
    1: "syrena",
}


def run_yolo(file_names: list[str]) -> pd.DataFrame:
    """
    Uruchamia model YOLO (Syrena/Maluch) na liście plików.

    file_names – lista nazw plików (bez ścieżek).
    Zwraca DataFrame z kolumnami DETECTIONS_COLUMNS.

    Założenia:
    - obrazy znajdują się w katalogu IMAGES_DIR,
    - YOLO_MODEL_PATH wskazuje na best.pt z treningu Syrena/Maluch.
    """

    model = YOLO(str(YOLO_MODEL_PATH))
    rows = []

    for fname in file_names:
        img_path = IMAGES_DIR / fname
        if not img_path.exists():
            continue

        results = model(source=str(img_path), verbose=False)

        for r in results:
            if r.boxes is None:
                continue

            for box in r.boxes:
                cls_idx = int(box.cls.item())
                score = float(box.conf.item())
                xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]

                class_norm = YOLO_IDX_TO_CLASS_NORM.get(cls_idx, None)
                class_raw = model.names.get(cls_idx, str(cls_idx))

                rows.append(
                    {
                        "file_name": fname,
                        "model": "yolo_syrena_maluch",
                        "prompt": "",
                        "class_raw": class_raw,
                        "class_norm": class_norm,
                        "score": score,
                        "x_min": xyxy[0],
                        "y_min": xyxy[1],
                        "x_max": xyxy[2],
                        "y_max": xyxy[3],
                    }
                )

    if not rows:
        return empty_detections_df()

    return pd.DataFrame(rows, columns=DETECTIONS_COLUMNS)

### Konfiguracja LVIS dla benchmarku

In [87]:
from pathlib import Path

"""
Konfiguracja LVIS dla benchmarku.

Wykorzystujemy słownik:
- outputs/csv/lvis_subjects_en_pl.csv

Struktura oczekiwana:
- file_name   – nazwa pliku (może być pełna ścieżka),
- subject_en  – etykieta LVIS (np. 'car_(automobile)', 'dog', 'street_sign'),
- subject_pl  – tłumaczenie na polski (tu nie jest używane w benchmarku detektorów).

UWAGA:
- brak bboxów i score → LVIS traktujemy jako „tagger obrazów”.
"""

LVIS_RAW_CSV = PROJECT_ROOT / "outputs" / "csv" / "lvis_subjects_en_pl.csv"

if not LVIS_RAW_CSV.exists():
    raise FileNotFoundError(
        f"Nie znaleziono pliku z wynikami LVIS: {LVIS_RAW_CSV.as_posix()}\n"
        "Sprawdź, czy ścieżka jest poprawna."
    )

df_lvis_raw = pd.read_csv(LVIS_RAW_CSV)

required_cols = {"file_name", "subject_en", "subject_pl"}
missing = required_cols - set(df_lvis_raw.columns)
if missing:
    raise ValueError(
        f"Brakuje kolumn w df_lvis_raw: {missing}\n"
        "Sprawdź strukturę pliku lvis_subjects_en_pl.csv."
    )

# normalizacja nazw plików: zawsze bierzemy samą nazwę (bez ścieżki)
df_lvis_raw["file_name_norm"] = df_lvis_raw["file_name"].apply(
    lambda x: Path(str(x)).name
)

"""
Mapowanie etykiet LVIS (subject_en) → Twoje class_norm.

Na razie przykładowe mapowania – rozszerzysz po obejrzeniu unikalnych subject_en.
"""

LVIS_LABEL_TO_CLASS_NORM = {
    # pojazdy
    "car_(automobile)": "samochod_inny",
    "freight_car": "samochod_inny",
    "bus_(vehicle)": "autobus",

    # zwierzęta
    "dog": "pies",
    # "domestic_cat": "kot",  # dopisz jeśli występuje

    # znaki / szyldy
    "traffic_light": "znak_drogowy",
    "stop_sign": "znak_drogowy",
    "street_sign": "znak_drogowy",
    "store_sign": "szyld_sklepowy",
    "shop_sign": "szyld_sklepowy",

    # architektura – do uzupełnienia po obejrzeniu danych
    # "church": "kosciol",
    # "tower": "zabytkowa_wieza",
    # "balcony": "balkon",
    # "shop_window": "witryna_sklepowa",
    # "store_window": "witryna_sklepowa",
}

### 3. Funkcja run_lvis (szkielet podmieniony)

In [88]:
def run_lvis(file_names: list[str]) -> pd.DataFrame:
    """
    Wykorzystuje słownik LVIS (lvis_subjects_en_pl.csv) jako źródło detekcji na poziomie obrazu.

    Logika:
    - df_lvis_raw: wiersz = (file_name, subject_en, subject_pl),
    - file_name_norm = tylko nazwa pliku (bez ścieżki),
    - filtrujemy tylko obrazy z listy file_names,
    - subject_en → class_raw,
    - subject_en → class_norm (słownik LVIS_LABEL_TO_CLASS_NORM),
    - bbox = 0.0 (placeholder), score = 1.0 (brak realnego score).

    file_names – lista nazw plików (bez ścieżek).
    """

    if df_lvis_raw.empty:
        return empty_detections_df()

    # filtr po znormalizowanej nazwie
    df_sel = df_lvis_raw[df_lvis_raw["file_name_norm"].isin(file_names)].copy()
    if df_sel.empty:
        return empty_detections_df()

    rows = []

    for _, row in df_sel.iterrows():
        fname = str(row["file_name_norm"])
        label_en = str(row["subject_en"])

        class_raw = label_en
        class_norm = LVIS_LABEL_TO_CLASS_NORM.get(label_en, None)

        rows.append(
            {
                "file_name": fname,
                "model": "lvis",
                "prompt": "",          # LVIS nie używa promptów tekstowych
                "class_raw": class_raw,
                "class_norm": class_norm,
                "score": 1.0,          # brak realnego score – stała wartość
                "x_min": 0.0,
                "y_min": 0.0,
                "x_max": 0.0,
                "y_max": 0.0,
            }
        )

    if not rows:
        return empty_detections_df()

    return pd.DataFrame(rows, columns=DETECTIONS_COLUMNS)

### 4. Funkcja run_owl_vit (szkielet)

In [89]:
def run_owl_vit(file_names: list[str]) -> pd.DataFrame:
    """
    Uruchamia model OWL-ViT (open-vocabulary) na liście plików.

    file_names – lista nazw plików (bez ścieżek).
    Zwraca DataFrame z kolumnami DETECTIONS_COLUMNS.

    TODO:
    - załadować model OWL-ViT (np. z transformers),
    - zdefiniować listę promptów (na podstawie PROMPTS_BY_CLASS),
    - dla każdego obrazu i promptu zebrać boxy i wyniki,
    - zapisać prompt i class_raw (= prompt lub nazwa klasy),
    - zmapować na class_norm.
    """
    df = empty_detections_df()
    # TODO: implementacja detekcji OWL-ViT
    return df

### 5. Funkcja run_detr (opcjonalnie, szkielet)

In [90]:
def run_detr(file_names: list[str]) -> pd.DataFrame:
    """
    Uruchamia wybrany model DETR na liście plików (opcjonalnie).

    file_names – lista nazw plików (bez ścieżek).
    Zwraca DataFrame z kolumnami DETECTIONS_COLUMNS.

    TODO:
    - załadować model DETR (torchvision / transformers),
    - uruchomić detekcję na każdym obrazie,
    - zmapować klasy na class_norm (np. car → samochod_inny).
    """
    df = empty_detections_df()
    # TODO: implementacja detekcji DETR
    return df

In [91]:
df_yolo.head()
df_yolo["class_norm"].value_counts()

class_norm
maluch    7
syrena    5
Name: count, dtype: int64

In [92]:
print("LVIS rows:", len(df_lvis_raw))
print("Unikalne file_name_norm w LVIS:", df_lvis_raw["file_name_norm"].nunique())
print("Przykład:", df_lvis_raw["file_name_norm"].head())
print("Przykład z file_list:", file_list[:5])

LVIS rows: 1072
Unikalne file_name_norm w LVIS: 74
Przykład: 0    0004.jpg
1    0004.jpg
2    0004.jpg
3    0004.jpg
4    0004.jpg
Name: file_name_norm, dtype: object
Przykład z file_list: ['0000.jpg', '000202.jpg', '000238.jpg', '0003.jpg', '000345.jpg']


In [93]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve()
EVAL_OUTPUT_DIR = PROJECT_ROOT / "outputs" / "detector_eval"

df_yolo = pd.read_csv(EVAL_OUTPUT_DIR / "detections_yolo_syrena_maluch.csv")
print("Wierszy:", len(df_yolo))
print(df_yolo["class_norm"].value_counts(dropna=False))

Wierszy: 12
class_norm
maluch    7
syrena    5
Name: count, dtype: int64
